In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
# Load the dataset
data = pd.read_csv('.\mbti_1.csv', encoding='latin1', error_bad_lines=False)  # Replace 'path_to_your_dataset.csv' with the actual path

# Display the first few rows of the dataset
print(data.head())

In [ ]:
# Splitting text posts into individual posts
data['posts'] = data['text'].str.split('|||')

# Optional: Remove commas from text posts using regular expressions
data['posts'] = data['posts'].apply(lambda x: re.sub(r',', '', x))

# Display the updated dataset with split posts
print(data.head())

tokenization + padding

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Splitting data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['posts'])

max_len = 100  # Adjust max length as needed
train_sequences = tokenizer.texts_to_sequences(train_data['posts'])
test_sequences = tokenizer.texts_to_sequences(test_data['posts'])

train_sequences_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Display some preprocessed data
print("Tokenized and Padded Sequences (Train):")
print(train_sequences_padded[:5])
print("Shape of Train Sequences Padded:", train_sequences_padded.shape)

encoding

In [ ]:
# Encode MBTI type labels
label_encoder = LabelEncoder()
label_encoder.fit(data['type'])

train_labels = label_encoder.transform(train_data['type'])
test_labels = label_encoder.transform(test_data['type'])

# Display encoded labels
print("Encoded Train Labels:", train_labels[:5])

model definition

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Define LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='softmax'))  # 16 output classes (MBTI types)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(train_sequences_padded, train_labels, epochs=10, batch_size=32, validation_split=0.2)